# Mod 3 Final Project

Jeremy Owens, Georgina Sampson

In [61]:
import pandas as pd
import sqlite3
import numpy as np
from api_keys import jo_api_key, gs_api_key
import matplotlib.pyplot as plt
import requests
import re
from bs4 import BeautifulSoup
import time
%matplotlib inline

In [3]:
crime_data = pd.read_csv('denver_data/crime_short.csv')
dispensary_data = pd.read_csv('denver_data/marijuana_active_business_licenses.csv')
# offense_data = pd.read_csv('denver_data/offense_codes.csv')

In [4]:
print(len(crime_data))
crime_data.head()

450000


,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0


In [5]:
crime_data.columns = [x.lower() for x in crime_data.columns]

In [6]:
crime_data = crime_data[crime_data.is_crime == 1]

In [7]:
crime_data['year'] = crime_data.reported_date.apply(lambda date: int(date.split()[0][-4:]))

In [8]:
offense_type_ids = list(crime_data.groupby('offense_type_id').count().index)

In [9]:
violent_list = [off for off in offense_type_ids if 'assault' in off] + \
                    [off for off in offense_type_ids if 'homicide' in off] + \
                    [off for off in offense_type_ids if 'robbery' in off] + \
                    [off for off in offense_type_ids if 'sex-aslt' in off] + \
                    ['agg-aslt-police-weapon', 'aslt-agg-police-gun']
violent_list.pop(violent_list.index('traf-vehicular-assault'))

drug_list = [off for off in offense_type_ids if 'drug' in off]

In [10]:
def gen_offense_type(offense):
    if offense in violent_list:
        return 'violent'
    elif offense in drug_list:
        return 'drug'
    else:
        return 'other'

In [11]:
crime_data['gen_offense_type'] = crime_data.offense_type_id.apply(lambda offense: gen_offense_type(offense))

In [12]:
crime_data_short = crime_data.drop(['incident_id', 'offense_code', 'offense_id', 'offense_code_extension', 'offense_type_id', 
                                    'offense_category_id', 'first_occurrence_date', 'last_occurrence_date', 
                                    'incident_address', 'geo_x', 'geo_y', 'geo_lon', 'geo_lat', 'district_id', 
                                    'precinct_id', 'is_crime', 'is_traffic', 'reported_date'], 
                                    axis=1)
crime_data_short.sample(10)

,neighborhood_id,year,gen_offense_type
194224,cole,2017,other
398820,lincoln-park,2014,violent
11896,sunnyside,2018,other
53565,montclair,2019,other
18278,cole,2018,other
331957,cbd,2016,other
39484,speer,2019,other
433166,whittier,2017,other
117389,hale,2015,other
115642,congress-park,2015,other


In [70]:
crime_data_short.sample(15)

,neighborhood_id,year,gen_offense_type
268178,montbello,2014,other
285789,capitol-hill,2014,other
330921,highland,2016,other
8191,lincoln-park,2018,violent
205719,civic-center,2017,other
71791,hale,2014,other
369892,auraria,2018,other
40972,hampden,2019,violent
216928,villa-park,2018,other
418884,sloan-lake,2017,other


In [13]:
dispensary_data.columns = [x.lower().replace(' ', '_') for x in dispensary_data.columns]
dispensary_data = dispensary_data[['business_file_number', 'license_type', 'facility_zip_code']]

In [14]:
dispensary_data.dropna(inplace=True)
dispensary_data['zip_code'] = dispensary_data.facility_zip_code.astype('int64')
dispensary_data.drop('facility_zip_code', inplace=True, axis=1)

In [15]:
stores_data = dispensary_data[dispensary_data.license_type.isin(['Retail Marijuana Store', 
                                                                 'Medical Marijuana Center'])]
print(len(stores_data))
stores_data.head(10)

345


,business_file_number,license_type,zip_code
0,2016-BFN-0007402,Retail Marijuana Store,80204
2,2018-BFN-0000461,Retail Marijuana Store,80239
4,2014-BFN-0004811,Retail Marijuana Store,80216
8,2013-BFN-1069615,Retail Marijuana Store,80210
9,2013-BFN-1069055,Retail Marijuana Store,80202
11,2016-BFN-0004809,Retail Marijuana Store,80223
12,2015-BFN-0002256,Medical Marijuana Center,80211
19,2018-BFN-0007446,Retail Marijuana Store,80205
20,2010-BFN-1045700,Medical Marijuana Center,80209
21,2010-BFN-1048389,Medical Marijuana Center,80216


In [16]:
store_count = stores_data.groupby('zip_code')['business_file_number'].count()
store_count

zip_code
80123     2
80127     2
80202    13
80203    18
80204    25
80205    22
80206     5
80207     6
80209     8
80210    35
80211    20
80212     6
80214     2
80216    45
80218     6
80219    21
80220    15
80222     9
80223    39
80224    10
80231     4
80232     4
80236     2
80237     2
80239    20
80247     2
80249     2
Name: business_file_number, dtype: int64

In [17]:
html_page = requests.get('http://www.city-data.com/zipmaps/Denver-Colorado.html')
soup = BeautifulSoup(html_page.content, 'html.parser')

In [42]:
zip_code_list = []
zip_codes = soup.find(class_="zip-codes").findAll('a', recursive=False)
for tag in zip_codes:
    zip_code_list.append(int(tag.text))
zip_code_list[:5]

[80012, 80014, 80110, 80111, 80123]

In [75]:
zip_neighbor_dict = {}
for zip_code in zip_code_list:
    html_page = requests.get('http://www.city-data.com/zips/{}.html'.format(zip_code))
    soup = BeautifulSoup(html_page.content, 'html.parser')
    neighborhoods = []
    for a in soup.find_all('a'):
        if 'neighborhood' in a.text:
            if a.text.split()[-1] == 'neighborhood':
                if len(a.text.split()) < 6:
                    if 'Central Business District' in a.text:
                        neighborhood = 'cbd'
                    elif '(' in a.text:
                        neighborhood = '-'.join(a.text[:(a.text.index('(') - 1)].split()).lower()
                    else:
                        neighborhood = '-'.join(a.text.split()[:-1]).lower()
                    neighborhoods.append(neighborhood)
    zip_neighbor_dict.update({zip_code: neighborhoods})
    time.sleep(0.5)
    
zip_neighbor_dict

{80012: ['aurora-district', 'city-center'],
 80014: ['cherry-creek-district', 'kennedy'],
 80110: ['college-view',
  'fort-logan',
  'harvey-park-south',
  'overland',
  'south-platte',
  'southwest',
  'university',
  'wellshire'],
 80111: ['kennedy'],
 80123: ['fort-logan', 'marston', 'southwest'],
 80202: ['cbd',
  'central-platte-valley',
  'cbd',
  'five-points',
  'highland',
  'lower-downtown',
  'north-capitol-hill',
  'union-station'],
 80203: ['alamo-placita',
  'capitol-hill',
  'civic-center',
  'five-points',
  'golden-triangle',
  'north-capitol-hill',
  'speer',
  'uptown'],
 80204: ['auraria',
  'baker',
  'barnum',
  'barnum-west',
  'cbd',
  'civic-center',
  'colfax',
  'cbd',
  'golden-triangle',
  'lincoln-park',
  'lower-downtown',
  'sheridan-boulevard',
  'sloan-lake',
  'southwest',
  'sun-valley',
  'union-station',
  'valverde',
  'villa-park'],
 80205: ['ballpark',
  'city-park',
  'city-park-west',
  'clayton',
  'cole',
  'curtis-park',
  'five-points',
  

In [ ]:
for key,value in zip_neighbor_dict:
    list_of_matches = []
    

# Working Zone

In [ ]:
print(len(crime_data))
crime_data.isna().sum()

In [ ]:
print(len(crime_data))
crime_data.sample(10)

In [74]:
html_page = requests.get('http://www.city-data.com/zips/{}.html'.format(80202))
soup = BeautifulSoup(html_page.content, 'html.parser')
neighborhoods = []
for a in soup.find_all('a'):
    if 'neighborhood' in a.text:
        if a.text.split()[-1] == 'neighborhood':
            if len(a.text.split()) < 6:
                if 'Central Business District' in a.text:
                    neighborhood = 'cbd'
                elif '(' in a.text:
                    neighborhood = '-'.join(a.text[:(a.text.index('(') - 1)].split()).lower()
                else:
                    neighborhood = '-'.join(a.text.split()[:-1]).lower()
                neighborhoods.append(neighborhood)
neighborhoods

['cbd',
 'central-platte-valley',
 'cbd',
 'five-points',
 'highland',
 'lower-downtown',
 'north-capitol-hill',
 'union-station']

In [ ]:
violent_data = crime_data[crime_data['gen_offense_type'] == 'violent']
drug_data = crime_data[crime_data['gen_offense_type'] == 'drug']

In [ ]:
offense_data.head(10)

In [ ]:
offense_data.groupby('OFFENSE_CATEGORY_ID')['OFFENSE_CODE'].count()

In [ ]:
print(len(dispensary_data))
dispensary_data.sample(10)

In [ ]:
dispensary_data.groupby('Trade Name').count()

In [ ]:
dispensary_data[dispensary_data['Trade Name'] == '1136 YUMA']